In [1]:
import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib
# import matplotlib.pyplot as plt
# import plotly.express as px
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
# import random

In [2]:
df = pd.read_csv(r'encuesta.csv', delimiter=r'\s+', engine='python', on_bad_lines='skip')
print(df.shape)
df.head()

(30007, 172)


,Q1A,Q1I,Q1E,Q2A,Q2I,Q2E,Q3A,Q3I,Q3E,Q4A,...,screensize,uniquenetworklocation,hand,religion,orientation,race,voted,married,familysize,major
0,4,28,3890,4,25,2122,2,16,1944,4,...,1,1,1,12,1,10,2,1,2,None
1,4,2,8118,1,36,2890,2,35,4777,3,...,2,1,2,7,0,70,2,1,4,None
2,3,7,5784,1,33,4373,4,41,3242,1,...,2,1,1,4,3,60,1,1,3,None
3,2,23,5081,3,11,6837,2,37,5521,1,...,2,1,2,4,5,70,2,1,5,biology
4,2,36,3215,2,13,7731,3,5,4156,4,...,2,2,3,10,1,10,2,1,4,Psychology


In [3]:
#Separacion de datos en distintos DFs para poder trabajar con ellos

preguntas = df.iloc[:, :126]    #Cuestionario del DASS

tipi = df.iloc[:, 131:141]      #10-items personality inventory

lapse = df.iloc[:, 126:131]     #columna de tiempo que se tarda en responder la encuesta

vlc = df.iloc[:, 141:157]       #palabras para evaluación cognitiva, dentro de contexto educativo y de atención al cuestionario

demograficos = df.iloc[:, 157:] #datos demográficos


In [4]:
#Separamos los Q y nos quedamos con QA que son las respuestas al cuestionario

listaA = []
for n in range(1, 43):
    numero = f'Q{n}I'
    listaA.append(numero)
QA = preguntas.drop(listaA, axis=1)

listaI = []
for n in range(1, 43):
    numero = f'Q{n}A'
    listaI.append(numero)
QE = QA.drop(listaI, axis=1)

listaE = []
for n in range(1, 43):
    numero = f'Q{n}E'
    listaE.append(numero)
QA=QA.drop(listaE,axis=1)



In [5]:
def separarQA(lista,df):
    lista_vacia=[]

    for n in lista:
        numero = f'Q{n}A'
        lista_vacia.append(numero)
        nuevo_QA=df[lista_vacia]

    return nuevo_QA

In [6]:
#Separamos los QA en el tipo de calculo que realizan

anxiety_list=[ 2, 4, 7, 9, 15, 19, 20, 23, 25, 28, 30, 36, 40, 41]
depression_list=[3, 5, 10, 13, 16, 17, 21, 24, 26, 31, 34, 37, 38, 42]
stress_list= [1, 6, 8, 11, 12, 14, 18, 22, 27, 29, 32, 33, 35, 39]

Anxiety=separarQA(anxiety_list,QA)
Depression=separarQA(depression_list,QA)
Stress=separarQA(stress_list,QA)

Anxiety.head(2)
print(Anxiety.shape,Depression.shape,Stress.shape)

(30007, 14) (30007, 14) (30007, 14)


In [7]:
#Añadimos a QA la columna de calculo de estrés/ansiedad/depresión
#el cálculo se realiza sumando los valores de las preguntas y restando 14
#(el cuestionario debería ser con valores de 0-3, pero al estar de 1-4 hacemos esto para un mejor calculo)
# el cálculo se realiza siguiendo las instrucciones de los datos oficiales del inventario de ansiedad de Beck

QA['Suma Estrés']=((Stress.sum(axis=1))-14)
QA['Suma Depresión']=((Depression.sum(axis=1))-14)
QA['Suma Ansiedad']=((Anxiety.sum(axis=1))-14)

QA.head(2)

,Q1A,Q2A,Q3A,Q4A,Q5A,Q6A,Q7A,Q8A,Q9A,Q10A,...,Q36A,Q37A,Q38A,Q39A,Q40A,Q41A,Q42A,Suma Estrés,Suma Depresión,Suma Ansiedad
0,4,4,2,4,4,4,4,4,2,1,...,4,1,2,4,3,4,4,40,27,34
1,4,1,2,3,4,4,3,4,3,2,...,3,4,2,2,1,2,2,27,24,17


In [8]:
valoracion=[]
for n in QA['Suma Depresión']:

    if n <=9:
        valoracion.append('Ninguna')
    elif n in range(10,14):
        valoracion.append('Muy leve')
    elif n in range(14,21):
        valoracion.append('Leve / Moderada')
    elif n in range(21,28):
        valoracion.append('Importante')
    elif n >= 28:
        valoracion.append('Grave')
    else: valoracion.append('Error de conteo')

QA['Valoracion Depresión']=(valoracion)



valoracion=[]
for n in QA['Suma Ansiedad']:

    if n <=7:
        valoracion.append('Ninguna')
    elif n in range(8,10):
        valoracion.append('Muy leve')
    elif n in range(10,15):
        valoracion.append('Leve / Moderada')
    elif n in range(15,20):
        valoracion.append('Importante')
    elif n >= 20:
        valoracion.append('Grave')
    else: valoracion.append('Error de conteo')

QA['Valoracion Ansiedad']=(valoracion)



valoracion=[]
for n in QA['Suma Estrés']:

    if n <=14:
        valoracion.append('Ninguna')
    elif n in range(15,19):
        valoracion.append('Muy leve')
    elif n in range(19,26):
        valoracion.append('Leve / Moderada')
    elif n in range(26,34):
        valoracion.append('Importante')
    elif n >= 34:
        valoracion.append('Grave')
    else: valoracion.append('Error de conteo')

QA['Valoracion Estrés']=(valoracion)

In [9]:
df=tipi

# Invertimos los tipis negativos y calculamos la media de los tipis para cada rasgo de personalidad

# df['TIPI6'] = 8 - df['TIPI6']
# df['TIPI2'] = 8 - df['TIPI2']
# df['TIPI8'] = 8 - df['TIPI8']
# df['TIPI4'] = 8 - df['TIPI4']
# df['TIPI10'] = 8 - df['TIPI10']

# df['Extraversión'] = df[['TIPI1', 'TIPI6']].mean(axis=1)
# df['Amabilidad'] = df[['TIPI2', 'TIPI7']].mean(axis=1)
# df['Responsabilidad'] = df[['TIPI3', 'TIPI8']].mean(axis=1)
# df['Neuroticismo'] = df[['TIPI4', 'TIPI9']].mean(axis=1)
# df['Apertura exp'] = df[['TIPI5', 'TIPI10']].mean(axis=1)

df.head(2)

,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,TIPI6,TIPI7,TIPI8,TIPI9,TIPI10
0,1,5,7,7,7,7,7,5,1,1
1,6,5,4,7,5,4,7,7,1,5


In [10]:
conteos = demograficos['major'].value_counts()

valores_menos_de_100 = conteos[conteos < 100].index

demograficos['major'] = demograficos['major'].apply(lambda x: 0 if x in valores_menos_de_100 else x)

demograficos['major'].unique()

array([None, 0, 'Psychology', 'Art', 'Business', 'psychology', 'Biology',
       'History', 'Education', 'Architecture', nan, 'Marketing',
       'Engineering', 'Law', 'English', 'Accounting', 'Dentistry',
       'Nursing', 'medicine', 'IT', 'Communication', 'english',
       'Economics', 'Physics', 'engineering', 'Finance', 'Science',
       'Pharmacy', 'Management', 'law', 'business', 'Medicine',
       'Chemistry', 'Medical', 'Mathematics', 'accounting', 'Accountancy',
       'Tourism'], dtype=object)

In [11]:
categorias = {
    'ciencias': ['Biology', 'Physics', 'Chemistry', 'Science', 'Mathematics'],
    'medicina': ['Medicine', 'Dentistry', 'Nursing', 'Pharmacy', 'Medical', 'medicine'],
    'psicologia': ['Psychology', 'psychology'],
    'empresariales': ['Business', 'Marketing', 'Finance', 'Management', 'Accounting', 'accounting', 'Accountancy', 'business'],
    'ingenieria': ['Engineering', 'engineering'],
    'humanidades': ['History', 'English', 'english', 'Law', 'law', 'Communication'],
    'arte': ['Art', 'Architecture'],
    'tecnología': ['IT'],
    'turismo':['Tourism'],
    '0': [None, 0]}

In [12]:

inverso_categorias = {}
for categoria, valores in categorias.items():
    for valor in valores:
        inverso_categorias[valor] = categoria

# Función de mapeo

def mapear_categoria(valor):
    return inverso_categorias.get(valor, '0')

# Aplicar la función de mapeo

demograficos['major'] = demograficos['major'].apply(mapear_categoria)

In [13]:
df_proba=pd.concat([QA, tipi, vlc, demograficos], axis=1)
df_proba=df_proba.drop(['screensize','uniquenetworklocation'],axis=1)
df_proba

,Q1A,Q2A,Q3A,Q4A,Q5A,Q6A,Q7A,Q8A,Q9A,Q10A,...,engnat,age,hand,religion,orientation,race,voted,married,familysize,major
0,4,4,2,4,4,4,4,4,2,1,...,2,16,1,12,1,10,2,1,2,0
1,4,1,2,3,4,4,3,4,3,2,...,1,16,2,7,0,70,2,1,4,0
2,3,1,4,1,4,3,1,3,2,4,...,2,17,1,4,3,60,1,1,3,0
3,2,3,2,1,3,3,4,2,3,3,...,1,13,2,4,5,70,2,1,5,0
4,2,2,3,4,4,2,4,4,4,3,...,2,19,3,10,1,10,2,1,4,psicologia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30002,2,1,1,1,2,2,1,2,1,1,...,1,27,1,4,1,60,1,1,3,psicologia
30003,3,4,2,2,2,4,4,2,1,3,...,1,16,1,1,1,70,2,1,3,0
30004,2,1,3,2,3,2,1,3,1,4,...,1,16,1,2,4,60,2,1,2,0
30005,3,4,3,4,3,4,4,4,3,4,...,2,21,1,10,0,10,2,1,4,0


In [14]:
encoder=LabelEncoder()

df_proba['major']=encoder.fit_transform(df_proba['major'])

In [15]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
import joblib

In [16]:
df_proba.columns

Index(['Q1A', 'Q2A', 'Q3A', 'Q4A', 'Q5A', 'Q6A', 'Q7A', 'Q8A', 'Q9A', 'Q10A',
       'Q11A', 'Q12A', 'Q13A', 'Q14A', 'Q15A', 'Q16A', 'Q17A', 'Q18A', 'Q19A',
       'Q20A', 'Q21A', 'Q22A', 'Q23A', 'Q24A', 'Q25A', 'Q26A', 'Q27A', 'Q28A',
       'Q29A', 'Q30A', 'Q31A', 'Q32A', 'Q33A', 'Q34A', 'Q35A', 'Q36A', 'Q37A',
       'Q38A', 'Q39A', 'Q40A', 'Q41A', 'Q42A', 'Suma Estrés', 'Suma Depresión',
       'Suma Ansiedad', 'Valoracion Depresión', 'Valoracion Ansiedad',
       'Valoracion Estrés', 'TIPI1', 'TIPI2', 'TIPI3', 'TIPI4', 'TIPI5',
       'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10', 'VCL1', 'VCL2', 'VCL3',
       'VCL4', 'VCL5', 'VCL6', 'VCL7', 'VCL8', 'VCL9', 'VCL10', 'VCL11',
       'VCL12', 'VCL13', 'VCL14', 'VCL15', 'VCL16', 'education', 'urban',
       'gender', 'engnat', 'age', 'hand', 'religion', 'orientation', 'race',
       'voted', 'married', 'familysize', 'major'],
      dtype='object')

In [17]:

df_stress=QA.drop(['Suma Estrés','Suma Depresión', 'Suma Ansiedad', 'Valoracion Depresión', 'Valoracion Ansiedad'],axis=1)

normalizador = MinMaxScaler()


print('--------------------------ESTRÉS-----------------------------')

df_stress['Valoracion Estrés'] = encoder.fit_transform(df_stress['Valoracion Estrés'])

X_stress = df_stress.drop('Valoracion Estrés', axis=1)
y_stress = df_stress['Valoracion Estrés']

X_stress = normalizador.fit_transform(X_stress)
joblib.dump(normalizador, 'normalizador_stress.pkl')

X_train_stress, X_test_stress, y_train_stress, y_test_stress = train_test_split(X_stress, y_stress, test_size=0.25, random_state=42)

y_train_stress = to_categorical(y_train_stress, num_classes=5)
y_test_stress = to_categorical(y_test_stress, num_classes=5)

model_stress = Sequential()
model_stress.add(Input(shape=(X_train_stress.shape[1],)))

model_stress.add(Dense(250, activation='relu'))
model_stress.add(Dense(225, activation='relu'))
model_stress.add(Dense(200, activation='relu'))
model_stress.add(Dense(175, activation='relu'))
model_stress.add(Dense(150, activation='relu'))
model_stress.add(Dense(125, activation='relu'))
model_stress.add(Dense(100, activation='relu'))
model_stress.add(Dense(75, activation='relu'))
model_stress.add(Dense(50, activation='relu'))
model_stress.add(Dense(25, activation='relu'))
model_stress.add(Dense(5, activation='softmax'))

model_stress.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_stress.fit(X_train_stress,
                y_train_stress,
                validation_data = (X_test_stress, y_test_stress), 
                epochs=15, 
                batch_size=32)

loss, accuracy = model_stress.evaluate(X_test_stress, y_test_stress)

print(f"Pérdida: {loss}, Precisión: {accuracy}")


--------------------------ESTRÉS-----------------------------
Epoch 1/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.6855 - loss: 0.6854 - val_accuracy: 0.9016 - val_loss: 0.2183
Epoch 2/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9051 - loss: 0.2284 - val_accuracy: 0.9468 - val_loss: 0.1313
Epoch 3/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9396 - loss: 0.1429 - val_accuracy: 0.8526 - val_loss: 0.3861
Epoch 4/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9468 - loss: 0.1264 - val_accuracy: 0.9355 - val_loss: 0.1490
Epoch 5/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9528 - loss: 0.1148 - val_accuracy: 0.9743 - val_loss: 0.0855
Epoch 6/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9669 - loss: 0.0894 - val_accuracy: 0.9733 - val_loss: 0.0585
Epoch 7/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9705 - loss: 0.0750 - val_accuracy: 0.9984 - val_loss: 0.0205
Epoch 8/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 6

In [18]:
df_anxiety=QA.drop(['Suma Depresión','Suma Ansiedad', 'Suma Estrés', 'Valoracion Depresión', 'Valoracion Estrés'],axis=1)

normalizador = MinMaxScaler()
encoder = LabelEncoder()

print('--------------------------ANSIEDAD-----------------------------')

df_anxiety['Valoracion Ansiedad'] = encoder.fit_transform(df_anxiety['Valoracion Ansiedad'])

X_anxiety = df_anxiety.drop('Valoracion Ansiedad', axis=1)
y_anxiety = df_anxiety['Valoracion Ansiedad']

X_anxiety = normalizador.fit_transform(X_anxiety)

X_train_anxiety, X_test_anxiety, y_train_anxiety, y_test_anxiety = train_test_split(X_anxiety, y_anxiety, test_size=0.25, random_state=42)

y_train_anxiety = to_categorical(y_train_anxiety, num_classes=5)
y_test_anxiety = to_categorical(y_test_anxiety, num_classes=5)

model_anxiety = Sequential()
model_anxiety.add(Input(shape=(X_train_anxiety.shape[1],)))

model_anxiety.add(Dense(250, activation='relu'))
model_anxiety.add(Dense(225, activation='relu'))
model_anxiety.add(Dense(200, activation='relu'))
model_anxiety.add(Dense(175, activation='relu'))
model_anxiety.add(Dense(150, activation='relu'))
model_anxiety.add(Dense(125, activation='relu'))
model_anxiety.add(Dense(100, activation='relu'))
model_anxiety.add(Dense(75, activation='relu'))
model_anxiety.add(Dense(50, activation='relu'))
model_anxiety.add(Dense(25, activation='relu'))
model_anxiety.add(Dense(5, activation='softmax'))

model_anxiety.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_anxiety.fit(X_train_anxiety, 
                  y_train_anxiety, 
                  epochs=15, 
                  validation_data = (X_test_anxiety, y_test_anxiety), 
                  batch_size=32)

loss, accuracy = model_anxiety.evaluate(X_test_anxiety, y_test_anxiety)

print('\n')
print(f"Pérdida: {loss}, Precisión: {accuracy}")


--------------------------ANSIEDAD-----------------------------
Epoch 1/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.7663 - loss: 0.5671 - val_accuracy: 0.8307 - val_loss: 0.4382
Epoch 2/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9369 - loss: 0.1552 - val_accuracy: 0.9175 - val_loss: 0.1822
Epoch 3/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9511 - loss: 0.1280 - val_accuracy: 0.9651 - val_loss: 0.0875
Epoch 4/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9657 - loss: 0.0884 - val_accuracy: 0.9203 - val_loss: 0.2291
Epoch 5/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9644 - loss: 0.0960 - val_accuracy: 0.9388 - val_loss: 0.1473
Epoch 6/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9740 - loss: 0.0746 - val_accuracy: 0.9896 - val_loss: 0.0334
Epoch 7/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9728 - loss: 0.0783 - val_accuracy: 0.9713 - val_loss: 0.0666
Epoch 8/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 

In [19]:
df_depression=QA.drop(['Suma Estrés', 'Suma Ansiedad','Suma Depresión', 'Valoracion Ansiedad', 'Valoracion Estrés'],axis=1)

normalizador = MinMaxScaler()
encoder = LabelEncoder()

print('--------------------------DEPRESIÓN-----------------------------')

df_depression['Valoracion Depresión'] = encoder.fit_transform(df_depression['Valoracion Depresión'])

X_depression = df_depression.drop('Valoracion Depresión', axis=1)
y_depression = df_depression['Valoracion Depresión']

X_depression = normalizador.fit_transform(X_depression)

X_train_depression, X_test_depression, y_train_depression, y_test_depression = train_test_split(X_depression, y_depression, test_size=0.25, random_state=42)

y_train_depression = to_categorical(y_train_depression, num_classes=5)
y_test_depression = to_categorical(y_test_depression, num_classes=5)

model_depression = Sequential()
model_depression.add(Input(shape=(X_train_depression.shape[1],)))

model_depression.add(Dense(250, activation='relu'))
model_depression.add(Dense(225, activation='relu'))
model_depression.add(Dense(200, activation='relu'))
model_depression.add(Dense(175, activation='relu'))
model_depression.add(Dense(150, activation='relu'))
model_depression.add(Dense(125, activation='relu'))
model_depression.add(Dense(100, activation='relu'))
model_depression.add(Dense(75, activation='relu'))
model_depression.add(Dense(50, activation='relu'))
model_depression.add(Dense(25, activation='relu'))
model_depression.add(Dense(5, activation='softmax'))

model_depression.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_depression.fit(X_train_depression, 
                     y_train_depression, 
                     epochs=15,
                     validation_data = (X_test_depression, y_test_depression), 
                     batch_size=32)

loss, accuracy = model_depression.evaluate(X_test_depression, y_test_depression)

print('\n')
print(f"Pérdida: {loss}, Precisión: {accuracy}")


--------------------------DEPRESIÓN-----------------------------
Epoch 1/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.7405 - loss: 0.5934 - val_accuracy: 0.9144 - val_loss: 0.1862
Epoch 2/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9228 - loss: 0.1846 - val_accuracy: 0.9453 - val_loss: 0.1416
Epoch 3/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9491 - loss: 0.1262 - val_accuracy: 0.9612 - val_loss: 0.0886
Epoch 4/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9489 - loss: 0.1270 - val_accuracy: 0.9529 - val_loss: 0.1104
Epoch 5/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9568 - loss: 0.1164 - val_accuracy: 0.9636 - val_loss: 0.0735
Epoch 6/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9652 - loss: 0.0910 - val_accuracy: 0.8694 - val_loss: 0.4033
Epoch 7/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9709 - loss: 0.0766 - val_accuracy: 0.9589 - val_loss: 0.0747
Epoch 8/15
704/704 ━━━━━━━━━━━━━━━━━━━━

In [20]:
model_stress.save('Q_stress.keras')
model_anxiety.save('Q_anxiety.keras')
model_depression.save('Q_depression.keras')